# Sistema de recomendación basado en memoria
- _Hecho por:_ Andre Pires
- _Correo:_ apt1007@alu.ubu.es
- _Fecha:_ 30 - 11 - 2025

## Introduccion
Como ya hemos visto en la asignatura los algoritmos de filtrado colaborativo agrupan a los usuarios en función del comportamiento y utilizan las características generales del grupo para recomendar artículos a un usuario objetivo. Los sistemas de recomendación colaborativos funcionan según el principio de que usuarios similares comparten intereses y gustos similares.

En el caso de esta practica deberemos hacer un filtro basado en memoria los cuales son extensiones de los clasificadores de los usuarios similares $k$ más cercanos porque intentan predecir el comportamiento de un usuario objetivo con respecto a un artículo determinado basándose en usuarios o conjuntos de artículos similares.

## Codigo y explicaciones
### Importación
En esta parte no tenemos mucho que explicar ya que no deja de ser una primera carga de archivos para poder emepzar a trabajr con ellos.

Tras tener las rutas, gracias a la libreria _pandas_, cargaremos los archivos para poder operar y trabajar con ellos como hemos hecho en otras practicas

In [60]:
import pandas as pd
import numpy as np

# Rutas
ruta_ratings = "dataset/ml-latest-small/ratings.csv"
ruta_movies  = "dataset/ml-latest-small/movies.csv"

# Cargar ratings
ratings = pd.read_csv(ruta_ratings)
print("ratings.head():")
display(ratings.head())

# Carga  peliculas
movies = pd.read_csv(ruta_movies)
print("movies.head():")
display(movies.head())


ratings.head():


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


movies.head():


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


### Matriz de utlidad
La matriz nos es util ya que nos ayuda a representar toda la información de ratings del sistema de recomendación. 

En las filas colocamos los usuarios, en las columnas las películas y en cada celda guardamos la puntuación que un usuario ha dado a una película. De esta forma convertimos el histórico de ratings en una estructura numérica que podemos tratar fácilmente con operaciones de álgebra lineal.

Ya dentro del codigo, hacemos nuestra matriz de utlidad usando las filas como id de los usuarios y como columnas como el id de las peliculas, poniento como intersección entre estas el rating que ha dado ese usuario a la pelicula.

In [61]:
# Matriz de utilidad
matriz_utilidad = ratings.pivot_table( index="userId", columns="movieId", values="rating").astype(float)
matriz_utilidad.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Similitud de productos
En este filtro colaborativo basado en productos vamos a utilizar la similitud del coseno para medir que tan parecidas son dos peliculas segun los ratings que han recibido de los usuarios.

En este punto podriamos calcular la similitud del coseno directamente con funciones de librerias como _sklearn_, que permiten obtener una matriz de similitud ítem-ítem de forma muy eficiente. Sin embargo, estas funciones requieren eliminar los valores NaN y normalmente se rellenan con 0, lo que implica asumir que las no-valoraciones se tratan como ceros. Siendo asi, preferiremos mantenerlo asi y hacer nuestro propio metodo. Ademas estas siblerias puede que tegan errores al la hora de calcular la similitud ya que en vez de permitirnos hacerlo con los vectores lo hace con valores unicos.

#### Explicacion 
En este caso aplicaremos la simulid a traves de la formula del coseno como, como ya se ha nombrado antes, pero antes de hacer el metodo como tal, vamos a entender un  poco que vamos a hacer.

Esta formula no mide la distancia geometrica, sino la orientación angular entre dos vectores. En nuestro contexto, esto lo pdemos aplicar ya que si Si dos vectores apuntan en direcciones muy similares en el espacio, se consideran muy parecidos en cuanto a su perfil de características.

La formula como tal es la siguiente:

<img  width="50%" src="./Formula_coseno.png">

En esta podemos encontrar por un lado en el numerador:
$$\sum_{u \in U} r_{u,i} \cdot r_{u,j}$$
Donde: 
- Se hace el producto escalar entre el vector de ratings del ítem $i$ y el vector de ratings del ítem $j$. Esto nos ayuda a medir la concordancia de los ratings. Si un usuario $u$ puntúa alto tanto a $i$ como a $j$, el producto $r_{u,i} \cdot r_{u,j}$ será alto y positivo, contribuyendo positivamente a la similitud

Por la parte del denominador encontramos:
$$\sqrt{\sum_{u \in U} r_{u,i}^2} \cdot \sqrt{\sum_{u \in U} r_{u,j}^2}$$
Donde:
- Es el producto de las magnitudes de los vectores de ratings de los items $i$ y $j$. Esto que el producto escalar se normaliza dividiéndolo por las magnitudes de los vectores asegurando asi que la similitud se base únicamente en el ángulo entre los vectores, ignorando la longitud de los vectores. Es decir, usuarios muy activos no dominan la métrica de similitud

In [62]:
def similitud_coseno(item1, item2, matriz_utilidad):
    
    ratings_item1 = matriz_utilidad[item1]
    ratings_item2 = matriz_utilidad[item2]

    usuarios_comunes = ratings_item1.notna() & ratings_item2.notna()
    n_usuarios_comunes = usuarios_comunes.sum()

    '''
    Significaria que no hay ningun usario en comun
    '''
    if n_usuarios_comunes == 0:
        return 0.0

    vector1 = ratings_item1[usuarios_comunes] 
    vector2 = ratings_item2[usuarios_comunes]

    producto_escalar = (vector1 * vector2).sum() # Producto escalar 
    norma_v1 = np.sqrt((vector1 ** 2).sum()) # Normal  vector 1
    norma_v2 = np.sqrt((vector2 ** 2).sum()) # Normal vector 2

    if norma_v1 == 0 or norma_v2 == 0:
        return 0.0

    '''
    Teniendo las variables bien establecidas y calculadas
    podemos hacer el la formula del coseno de manera
    muy sencila
    '''
    coseno = producto_escalar / (norma_v1 * norma_v2)

    return float(coseno)

### Similitud
A partir de la función de similitud del coseno item–item, definimos ahora un procedimiento para obtener las peliculas mas parecidas a una pelicula dada. Para ello, fijamos una pelicula objetivo y calculamos su similitud con el resto de peliculas de la matriz de utilidad. Después ordenamos estas similitudes de mayor a menor y seleccionamos las $k$ peliculas con valor mas alto. Esta lista de vecinos mas cercanos será la base para, mas adelante, poder predecir ratings y generar recomendaciones

In [63]:
def simlitud_peliculas(id_pelicula_objetivo, matriz_utilidad, df_peliculas):

    if id_pelicula_objetivo not in matriz_utilidad:
        raise ValueError("La pelicula no esta en la matriz de utilidad")
    
    similitudes = [] # Lista para guardar la id de la pelicula junto con su similitud
    for id_pelicula in matriz_utilidad.columns:
        if id_pelicula == id_pelicula_objetivo: # Esto lo hacemos para saltarse la propia pelicula como tal
            continue
        similitud = similitud_coseno(id_pelicula_objetivo, id_pelicula, matriz_utilidad)
        similitudes.append((id_pelicula, similitud))

    similitudes.sort(key=lambda x: x[1], reverse=True) # Ordenamos de mayor a menor

    '''
    Ahora, para poder operar en cualquier sistema aunque no tenga 
    muchos recursos, continuaremos calculando y trabajando con las
    10 mas parecidas
    '''
    mas_parecidad = similitudes[:10]

    '''
    Vamos a guradar y operar con un Dataframe con los resultados 
    concretos 
    '''
    resultados = []

    for movie_id, sim in mas_parecidad:
        
        fila_peli = df_peliculas[df_peliculas["movieId"] == movie_id] # Buscamos el titulo

        if not fila_peli.empty:
            titulo = fila_peli["title"].values[0]
        else:
            titulo = "Título no encontrado"

        resultados.append({"ID pelicula": movie_id, "tiulo": titulo, "similaridad": sim})

        resultados_df = pd.DataFrame(resultados)

    return resultados_df

### Pequeñas pruebas
Vamos a hacer una pequeñas pruebas para comprobar que se muestra y calcula todo de manera correcta.

Primero vamos a comprobar que las similitudes utilizando la formula del coseno se calculan, es decir, nos da un da estre -1 y 1; y se muestran de manera correcta

In [64]:
import random

idPelicula1 = random.randint(0, len(movies) - 1) # Eleccion de ID aleatorio
idPelicula2 = random.randint(0, len(movies) - 1) # Eleccion de ID aleatorio

pelicula1 = movies["movieId"].iloc[idPelicula1] # ID de la pelicula 1 para comprobar
pelicula2 = movies["movieId"].iloc[idPelicula2] # ID de la pelicula 2 para comprobar

print("movieId 1:", pelicula1, "-", movies.loc[movies["movieId"] == pelicula1, "title"].values[0])
print("movieId 2:", pelicula2, "-", movies.loc[movies["movieId"] == pelicula2, "title"].values[0])

sim = similitud_coseno(pelicula1, pelicula2, matriz_utilidad)
print("Similitud:", sim)


movieId 1: 36276 - Hidden (a.k.a. Cache) (Caché) (2005)
movieId 2: 63826 - Splinter (2008)
Similitud: 0.0


Tras ver que la similitud del coseno se calcula de forma correcta, vamos con algnas pruebas que, tras calcular la similitud con todas las peliculas, nos deberia de mostrar las 10 mas parecidas, y que por tanto, deberan tener un mayor puntuaje en la similitud de coseno

In [65]:
idPelicula = random.randint(0, len(movies) - 1) # Eleccion de ID aleatorio

movie_id_objetivo = int(movies["movieId"].iloc[idPelicula]) # Seleccion de pelicula 
print("movieId objetivo:", movie_id_objetivo)
print("Título:", movies["title"].iloc[idPelicula])

similares_df = simlitud_peliculas(movie_id_objetivo, matriz_utilidad, movies)

display(similares_df)


movieId objetivo: 4221
Título: Necessary Roughness (1991)


,ID pelicula,tiulo,similaridad
0,170,Hackers (1995),1.0
1,196,Species (1995),1.0
2,252,I.Q. (1994),1.0
3,288,Natural Born Killers (1994),1.0
4,333,Tommy Boy (1995),1.0
5,370,Naked Gun 33 1/3: The Final Insult (1994),1.0
6,376,"River Wild, The (1994)",1.0
7,434,Cliffhanger (1993),1.0
8,507,"Perfect World, A (1993)",1.0
9,516,Renaissance Man (1994),1.0


### Prediccion
Una vez que tenemos definida la similitud entre productos mediante el coseno, podemos utilizarlo para estimar el rating que un usuario daria a una pelicula que todavia no ha valorado como nos pide la practica. Esta seria la idea central del filtro colaborativo basado en items como hemos explicado con anterioridad.

Para ello, tomamos todas las peliculas que el usuario $u$ si ha valorado y para cada una de esas peliculas calculamos la similitud del coseno con la pelicula objetivo. Despues, ordenamos esas peliculas por similitud y nos quedamos con los $k$ vecinos mas similares para despues calcular el rating predicho como una media ponderada de los ratings de esos vecinos, donde los pesos son las similitudes. En relaciuon a esto, en los apuntes encontramos la formual que se usar para hacer lo explciado:

<img  width="25%" src="./Formula_prediccion.png">

Donde:
- ${r}_{u,j}$ es el rating predicho del usuario $u$ para la película $j$
- $r_{u,i}$ es el rating real que el usuario $u$ ha dado a la película $i$
- $N_k(u, j)$ es el conjunto de las $k$ películas más similares a $j$ que han sido valoradas por $u$
- ${sim}(i, j)$ es la similitud del coseno entre las películas $i$ y $j$

Ademas, con el uso de $k$ vecinos mas cercanos reduce el coste computacional, ya que solo trabajamos con los items mas informativos y hace que podamos ejecutarlo en otros ordenados con una especificaciones menores al mio.

In [66]:
def prediccion_raitng_item(user_id, movie_id, matriz_utilidad):
    '''
    Compoprbaciones previas antes de empezar
    para evitar errores futuros
    '''
    if user_id not in matriz_utilidad:
        raise ValueError("El ID del usuario no esta en la matriz de utilidad")
    if movie_id not in matriz_utilidad.columns:
        raise ValueError("El ID de la pelicula no esta en la matriz de utilidad")
    
    rating_actual = matriz_utilidad.loc[user_id, movie_id] # Nos sirve para devolver el valor real en caso de que no sea NaN
    if not np.isnan(rating_actual):
        return rating_actual
    
    fila_user = matriz_utilidad.loc[user_id]
    peliculas_valoradas = fila_user.dropna().index.tolist() # Peliculas que si ha valorado

    # Esto solo se aplicaria en caso de que el usuario no haya valorado ninguna pelicula
    if peliculas_valoradas == 0:
        return float(np.nanmean(matriz_utilidad.values)) # Devolvemos la media global
    
    vecinos = []
    for movie_id_vecino in peliculas_valoradas:
        if movie_id_vecino == movie_id:
            continue
        similitud = similitud_coseno(movie_id, movie_id_vecino, matriz_utilidad)
        rating_vecino = fila_user[movie_id_vecino]
        vecinos.append((movie_id_vecino, similitud, rating_vecino))
    
    # Ordenamos de mayor a menor
    vecinos.sort(key=lambda x: x[1], reverse=True) 

    # Trabajaremos con k vecinos
    vecinos_k = vecinos[:10]

    '''
    Ahora, teniendo ya todos los datos necesarios, vamos a ir calculando
    y haciendo las operacion pertinenetes para calcular el numerador
    y el denominador de la formual que vamos a usar que es la que esta
    explciado y analizada en el parrafo anterior 
    '''
    numerador = 0.0
    denominador = 0.0

    for movie_id_vecino, similitud, rating_vecino in vecinos_k:
        numerador == numerador + similitud * rating_vecino
        denominador == denominador + abs(similitud)
    
    # Si no hay vecino utiles, devolvemos la media
    if denominador == 0.0:
        return float(fila_user.mean())
    
    return float(numerador/denominador)

### Prubas finales y creacion de usuario nuevo
Como prueba final del funcionamiento del filtro, crearemos un usuario nuevo que no existia en el conjunto de datos original. A este usuario le asignamos valoraciones aleatorias sobre un conjunto reducido de peliculas, y reconstruimos la matriz de utilidad para incorporar su informacion

A partir de este historial, seleccionamos una pelicula que el nuevo usuario no ha valorado y utilizamos el modelo que hemos creado para predecir que rating le corresponderia. Esta estimacion se basara en las peliculas similares que si ha visto el usuario y en sus puntuaciones sobre ellas como hemos explicado antes

#### Creamos el usuario

In [67]:
nuevo_user_id = ratings["userId"].max() + 1 # Eleccion de un nuevo usuario

peliculas_mostradas = movies[movies["movieId"].isin(matriz_utilidad.columns)].sample(10, random_state=42)
display(peliculas_mostradas[["movieId", "title"]])


'''
Recorremos las peliculas y les damos un rating aleatorio
'''
registros = []

for indice, fila in peliculas_mostradas.iterrows():
    movie_id = int(fila["movieId"])
    rating_random = random.randint(1, 5)

    registro = {
        "userId": nuevo_user_id,
        "movieId": movie_id,
        "rating": rating_random,
    }

    registros.append(registro)

ratings_nuevo_usuario = pd.DataFrame(registros)
print("Ratings:")
display(ratings_nuevo_usuario)

,movieId,title
1146,1500,Grosse Pointe Blank (1997)
8523,114662,American Sniper (2014)
3908,5490,The Big Bus (1976)
1609,2151,"Gods Must Be Crazy II, The (1989)"
9159,148424,Chi-Raq (2015)
119,146,"Amazing Panda Adventure, The (1995)"
4610,6868,Wonderland (2003)
1459,1982,Halloween (1978)
7630,87485,Bad Teacher (2011)
2375,3152,"Last Picture Show, The (1971)"


Ratings:


,userId,movieId,rating
0,611,1500,1
1,611,114662,2
2,611,5490,1
3,611,2151,5
4,611,148424,4
5,611,146,5
6,611,6868,3
7,611,1982,3
8,611,87485,2
9,611,3152,5


#### Reconstruimos la matriz de utilidad

In [68]:
# Añadimos las nuevas valoraciones
ratings_extendido = pd.concat([ratings, ratings_nuevo_usuario], ignore_index=True)

matriz_utilidad_extendida = ratings_extendido.pivot_table(
    index="userId",
    columns="movieId",
    values="rating"
).astype(float)

# Mostarmos la cantidad de filas y columnas para ver la direfencia
print("Matriz utilidad original:", matriz_utilidad.shape)
print("Matriz utilidad extendida:", matriz_utilidad_extendida.shape)

Matriz utilidad original: (610, 9724)
Matriz utilidad extendida: (611, 9724)


#### Eleccion y estimacion de rating de pelicula

In [69]:
pelis_valoradas_nuevo = ratings_nuevo_usuario["movieId"].tolist() # Peliculas valoradas por el nuevo usuario
todas_pelis_extentes = matriz_utilidad_extendida.columns.tolist()

peliculas_no_valoradas_nuevo = [m for m in todas_pelis_extentes if m not in pelis_valoradas_nuevo] # Peliculas que el nuevo usuario no ha valorado

# Elegimos una pelicula objetivo
pelicula_objetivo = peliculas_no_valoradas_nuevo[random.randint(0, len(peliculas_no_valoradas_nuevo))]
titulo_objetivo = movies.loc[movies["movieId"] == pelicula_objetivo, "title"].values

# Predecimos el reting
rating_predicho = prediccion_raitng_item(
    user_id = nuevo_user_id,
    movie_id = pelicula_objetivo,
    matriz_utilidad = matriz_utilidad_extendida,
)

print("Título:", titulo_objetivo)
print("Rating predicho para el nuevo usuario y la película objetivo:", rating_predicho)

Título: ['Fortress (1992)']
Rating predicho para el nuevo usuario y la película objetivo: 3.1
